In [127]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv


In [128]:
#Preparing the data
train_df=pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')

In [129]:
train_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
application_sample1=train_df.loc[train_df.TARGET==1].sample(frac=0.1,replace=False)
print('label 1 sample size',str(application_sample1.shape[0]))
application_sample0=train_df.loc[train_df.TARGET==0].sample(frac=0.1,replace=False)
print('label 0 sample size',str(application_sample0.shape[0]))
application=pd.concat([application_sample1,application_sample0],axis=0).sort_values('SK_ID_CURR')

label 1 sample size 2482
label 0 sample size 28269


In [131]:
application.shape

(30751, 122)

In [132]:
categorical_list=[]
numerical_list=[]
for i in application.columns.to_list():
    if application[i].dtype=='object':
        categorical_list.append(i)
    else:
        numerical_list.append(i)
print(f'Number of Categorical Features in list is/are {len(categorical_list)}')
print(f'--------------------------------------------------------------------------------------------')
print(f'Number of Numerical Features in list is/are {len(numerical_list)}')

Number of Categorical Features in list is/are 16
--------------------------------------------------------------------------------------------
Number of Numerical Features in list is/are 106


In [133]:
#Imputing Missing Values in Numerical Columns
from sklearn.impute import SimpleImputer
application[numerical_list]=SimpleImputer(strategy='median').fit_transform(application[numerical_list])
application[numerical_list]

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
7,100010.0,0.0,0.0,360000.0,1530000.0,42075.0,1530000.0,0.003122,-18850.0,-449.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,100014.0,0.0,1.0,112500.0,652500.0,21177.0,652500.0,0.022800,-10197.0,-679.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12,100016.0,0.0,0.0,67500.0,80865.0,5881.5,67500.0,0.031329,-13439.0,-2717.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
14,100018.0,0.0,0.0,189000.0,773680.5,32778.0,679500.0,0.010006,-14583.0,-203.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25,100030.0,0.0,0.0,90000.0,225000.0,11074.5,225000.0,0.028663,-19334.0,-3494.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307475,456215.0,1.0,1.0,144000.0,1303200.0,46809.0,1125000.0,0.007330,-13416.0,-2405.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.0
307476,456216.0,0.0,2.0,135000.0,1762110.0,48456.0,1575000.0,0.019689,-13790.0,-1878.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
307486,456230.0,0.0,1.0,292500.0,355536.0,18283.5,270000.0,0.072508,-16010.0,-1185.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
307507,456252.0,0.0,0.0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775.0,365243.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [134]:
del train_df;gc.collect()
application=pd.get_dummies(application,drop_first=True)
application.shape

(30751, 228)

In [135]:
X=application.drop(['SK_ID_CURR','TARGET'],axis=1)
y=application['TARGET']
feature_name=X.columns.to_list()

In [136]:
def cor_selector(X,y):
    cor_list=[]
    for i in X.columns.tolist():
        cor=np.corrcoef(X[i],y)[0,1]
        cor_list.append(cor)
    cor_list=[0 if np.isnan(i) else i for i in cor_list]    
    cor_feature=X.iloc[:,np.argsort(np.abs(cor_list))[-100:]].columns.tolist()
    cor_support=[True if i in cor_feature else False for i in feature_name]
    return cor_support,cor_feature

In [137]:
cor_support,cor_feature=cor_selector(X,y)
print(str(len(cor_feature)),'Selected Features')

100 Selected Features


In [138]:
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.preprocessing import MinMaxScaler
X_transformed=MinMaxScaler().fit_transform(X)
chi_selector=SelectKBest(chi2,k=100)
chi_selector.fit_transform(X_transformed,y)


array([[0.38925101, 0.41772152, 0.03597082, ..., 0.        , 0.        ,
        0.        ],
       [0.15923905, 0.17088608, 0.30937131, ..., 0.        , 0.        ,
        0.        ],
       [0.009401  , 0.00632911, 0.42787079, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.08139828, 0.06329114, 1.        , ..., 0.        , 1.        ,
        0.        ],
       [0.05885947, 0.05063291, 0.34221605, ..., 0.        , 0.        ,
        1.        ],
       [0.16583508, 0.15189873, 0.062091  , ..., 0.        , 1.        ,
        0.        ]])

In [139]:
chi_support=chi_selector.get_support()
chi_feature=X.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)),'Selected features')

100 Selected features


In [140]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector=RFE(estimator=LogisticRegression(),n_features_to_select=100,step=10,verbose=5)
rfe_selector.fit(X_transformed,y)

Fitting estimator with 226 features.
Fitting estimator with 216 features.
Fitting estimator with 206 features.
Fitting estimator with 196 features.
Fitting estimator with 186 features.
Fitting estimator with 176 features.
Fitting estimator with 166 features.
Fitting estimator with 156 features.
Fitting estimator with 146 features.
Fitting estimator with 136 features.
Fitting estimator with 126 features.
Fitting estimator with 116 features.
Fitting estimator with 106 features.


RFE(estimator=LogisticRegression(), n_features_to_select=100, step=10,
    verbose=5)

In [141]:
rfe_support=rfe_selector.get_support()
rfe_feature=X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)),'Selected Features')

100 Selected Features


In [142]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
embedded_lr_selector=SelectFromModel(LogisticRegression(penalty='l2'),threshold='1.25*median')
embedded_lr_selector.fit(X_transformed,y)


SelectFromModel(estimator=LogisticRegression(), threshold='1.25*median')

In [143]:
embedded_lr_support=embedded_lr_selector.get_support()
embedded_lr_feature=X.loc[:,embedded_lr_support].columns.tolist()
print(str(len(embedded_lr_feature)),'Selected Features')

91 Selected Features


In [144]:
from sklearn.ensemble import RandomForestClassifier
embedded_rf_selector=SelectFromModel(estimator=RandomForestClassifier(n_estimators=100),threshold='1.25*median')
embedded_rf_selector.fit(X,y)

SelectFromModel(estimator=RandomForestClassifier(), threshold='1.25*median')

In [145]:
embedded_rf_support=embedded_rf_selector.get_support()
embedded_rf_feature=X.iloc[:,embedded_rf_support].columns.tolist()
print(str(len(embedded_rf_feature)),'Selected Feature')

95 Selected Feature


In [150]:
pd.set_option('display.max_rows',None)
feature_selection_df=pd.DataFrame({'Feature':feature_name,'Pearson Coefficient':cor_support,'Chi2':chi_support,'RFE':rfe_support,'Logistic Regression':embedded_lr_support,'Random Forest':embedded_rf_support})
feature_selection_df['Total']=np.sum(feature_selection_df,axis=1)
feature_selection_df=feature_selection_df.sort_values(['Total','Feature'],ascending=False)
feature_selection_df.index=range(1,len(feature_selection_df)+1)
feature_selection_df.head(100)
